In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

c:\Users\Максим\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# считаем базу Часто задаваемых вопросов RUTUBE 
faq_1 = pd.read_excel(r"C:\Users\Максим\Desktop\work\main bot 2.0\rutube_helper_train\rag\rag\01_База_знаний.xlsx")
faq_2 = pd.read_excel(r"C:\Users\Максим\Desktop\work\main bot 2.0\rutube_helper_train\rag\rag\02_Реальные_кейсы.xlsx")
path = r'C:\Users\Максим\Desktop\work\main bot 2.0\locmodels'
# С помощью модели извлечения embbeddings из текста получим embbeddings для всех вопросов из FAQ.
# Веса модели можно найти по ссылке: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# model.save(path)
model = SentenceTransformer(path)


c:\Users\Максим\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [26]:
faq_1

,Тема,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Почему могут отключить монетизацию на видео и ...,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Авторское право,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
...,...,...,...,...,...
321,Загрузка видео,Какую категорию выбрать для видео?,Категория должна соответствовать содержанию ви...,ВИДЕО,Загрузка видео
322,Загрузка видео,Почему изменилась категория у моего видео?,Если при загрузке видео вы выбирали одну катег...,МОДЕРАЦИЯ,Смена категории/возрастные ограничения
323,Загрузка видео,Как удалить видео?,"Войдите в профиль на RUTUBE, перейдите в разде...",ВИДЕО,Загрузка видео
324,Загрузка видео,Как добавить тайм-коды под видео?,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...,ВИДЕО,Загрузка видео


In [10]:
faq_2 = faq_2[['Вопрос пользователя', 'Ответ сотрудника', 'Классификатор 1 уровня', 'Классификатор 2 уровня']]
faq_2

,Вопрос пользователя,Ответ сотрудника,Классификатор 1 уровня,Классификатор 2 уровня
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,МОДЕРАЦИЯ,Отклонение/блокировка видео
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
...,...,...,...,...
796,Добрый день! Как добавить таймкоды в видео?,Характеристики эпизодов:\n\nОпределяются по та...,ВИДЕО,Загрузка видео
797,Как добавить временные интервалы,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...,ВИДЕО,Загрузка видео
798,В каком формате добавляются таймкоды?,Формат: минуты:секунды или часы:минуты:секунды...,ВИДЕО,Загрузка видео
799,Как мне правильно указать тайм-код?,Можете указать минуты:секунды или часы:минуты:...,ВИДЕО,Загрузка видео


In [17]:
dt = dict(zip(['Вопрос пользователя', 'Ответ сотрудника', 'Классификатор 1 уровня', 'Классификатор 2 уровня'], 
         ['Вопрос из БЗ', 'Ответ из БЗ', 'Классификатор 1 уровня', 'Классификатор 2 уровня']))

In [20]:
faq_2 =faq_2.rename(columns=dt)
faq_2.head()

,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,МОДЕРАЦИЯ,Отклонение/блокировка видео
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [25]:
all_data = pd.concat([faq_1[['Вопрос из БЗ', 'Ответ из БЗ', 'Классификатор 1 уровня', 'Классификатор 2 уровня']], faq_2], 
           names=['Вопрос из БЗ', 'Ответ из БЗ', 'Классификатор 1 уровня', 'Классификатор 2 уровня'], )

all_data

,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
...,...,...,...,...
796,Добрый день! Как добавить таймкоды в видео?,Характеристики эпизодов:\n\nОпределяются по та...,ВИДЕО,Загрузка видео
797,Как добавить временные интервалы,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...,ВИДЕО,Загрузка видео
798,В каком формате добавляются таймкоды?,Формат: минуты:секунды или часы:минуты:секунды...,ВИДЕО,Загрузка видео
799,Как мне правильно указать тайм-код?,Можете указать минуты:секунды или часы:минуты:...,ВИДЕО,Загрузка видео


In [27]:
faq_embeddings = model.encode(all_data['Вопрос из БЗ'].values)

In [33]:
faq_embeddings

array([[ 0.02049591,  0.00107754, -0.03493093, ...,  0.00925678,
         0.02923471, -0.08343392],
       [-0.04082318,  0.07071434,  0.03544611, ...,  0.07237033,
         0.03899818, -0.08159328],
       [-0.02075859,  0.07476   ,  0.04709725, ...,  0.05220534,
         0.01472712, -0.06279585],
       ...,
       [ 0.03655535,  0.08661007,  0.03833509, ...,  0.07916178,
         0.03937214, -0.08993171],
       [ 0.01129624,  0.04978478,  0.01566251, ...,  0.10775201,
         0.02200677, -0.02993451],
       [ 0.01148334,  0.01744441,  0.0138088 , ...,  0.11304776,
         0.00254555, -0.01203004]], dtype=float32)

In [29]:
db_params = {
    "dbname": "testdb",  # Название базы данных
    "user": "postgres",         # Имя пользователя
    "password": "mano1940",     # Пароль пользователя
    "host": "localhost",             # Адрес сервера (или IP)
    "port": 5433                     # Порт подключения
}

In [ ]:
import psycopg2

# Подключение к базе данных
conn = psycopg2.connect(**db_params)
conn.autocommit = True
cur = conn.cursor()

# Активируем расширение vector
# try:
#     cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
#     print("Расширение vector успешно активировано.")
# except Exception as e:
#     print(f"Ошибка активации расширения vector: {e}")

# Создание таблицы
try:
    create_table_query = """
    CREATE TABLE IF NOT EXISTS knowledge (
        id SERIAL PRIMARY KEY,              -- Уникальный идентификатор
        "Вопрос" TEXT,                      -- Столбец "Вопрос"
        "Ответ" TEXT,                       -- Столбец "Ответ"
        "Классификатор_1" TEXT,              -- Столбец "Классификатор_1"
        "Классификатор_2" TEXT,              -- Столбец "Классификатор_2"
        "Вектор" VECTOR(384)                -- Столбец "Вектор" с размерностью 384
    );
    """
    cur.execute(create_table_query)
    print("Таблица успешно создана.")
except Exception as e:
    print(f"Ошибка создания таблицы: {e}")

# Закрытие подключения
cur.close()
conn.close()


In [ ]:
import pandas as pd
import psycopg2

# Предполагается, что у вас есть два DataFrame: основной и с векторами.
# Пример создания DataFrame для демонстрации:
main_df = all_data[['Вопрос из БЗ', 'Ответ из БЗ', 'Классификатор 1 уровня', 'Классификатор 2 уровня']]

# Второй DataFrame, содержащий векторы размерности 384
vector_df = pd.DataFrame(faq_embeddings)

# Подключение к базе данных
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Параметризованный запрос для вставки данных в таблицу
insert_query = """
INSERT INTO knowledge ("id", "Вопрос", "Ответ", "Классификатор_1", "Классификатор_2", "Вектор")
VALUES (%s, %s, %s, %s, %s, %s);
"""

# Преобразование и объединение данных
for index, row in main_df.iterrows():
    # Извлечение вектора для текущей строки
    vector = vector_df.iloc[index].values
    vector_str = f"[{', '.join(map(str, vector))}]"  # Преобразование вектора в строковое представление для PostgreSQL

    # Параметры для вставки в таблицу
    data_tuple = (
        index+1,
        row["Вопрос из БЗ"],           # Вопрос
        row["Ответ из БЗ"],            # Ответ
        row["Классификатор 1 уровня"],  # Классификатор_1
        row["Классификатор 2 уровня"],  # Классификатор_2
        vector_str               # Вектор
    )

    try:
        # Выполнение команды вставки данных
        cur.execute(insert_query, data_tuple)
    except Exception as e:
        print(f"Ошибка при вставке данных: {e}")
        conn.rollback()

# Сохранение изменений
conn.commit()

# Закрытие соединения
cur.close()
conn.close()

print("Данные успешно вставлены в таблицу.")
